In [ ]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import random

%matplotlib inline

In [3]:
import sys

sys.path.append("./")
sys.path.append("../code/")
sys.path.append("./code/")

# Data Loader

In [4]:
from bps_numerical.preprocessing import merge_gene_phenotype, standardize_gene_data

In [5]:
CSV_GENE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/gen.csv"

CSV_PHENOTYPE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/meta.csv"

In [6]:
# df_merged = merge_gene_phenotype(standardize_gene_data(CSV_GENE), CSV_PHENOTYPE)

In [7]:
df_genes = standardize_gene_data(CSV_GENE)

2022-07-01 15:07:49.214 | INFO     | bps_numerical.preprocessing:standardize_gene_data:27 - Standardizing gene data into proper format.


In [8]:
df_genes.head()

,Sample,ENSMUSG00000099250,ENSMUSG00000064339,ENSMUSG00000099021,ENSMUSG00000064351,ENSMUSG00000065037,ENSMUSG00000064337,ENSMUSG00000100862,ENSMUSG00000029368,ENSMUSG00000064370,...,ENSMUSG00000013367,ENSMUSG00000008153,ENSMUSG00000013083,ENSMUSG00000013155,ENSMUSG00000012819,ENSMUSG00000010307,ENSMUSG00000011837,ENSMUSG00000012889,ENSMUSG00000001642,ENSMUSG00000006019
1,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,4.565105,4.486067,4.22359,4.137298,3.773993,3.86423,3.093246,4.149449,3.38502,...,-0.748711,0.514245,-0.834259,-0.637977,-1.04317,0.360869,0.374517,-0.665341,0.748225,0.347033
2,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,0.563454,3.140386,0.778641,4.384875,0.745577,3.01786,4.037345,4.180711,4.089054,...,-0.15882,0.151228,-1.032761,-0.42141,-1.151805,0.205813,0.746601,-0.813002,0.670417,0.730899
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,3.968279,3.490494,3.984012,3.721062,3.526439,2.489441,2.644076,4.093958,2.904823,...,-0.599768,-0.157375,-1.130212,-0.677432,-1.094534,0.10989,0.389537,-0.571371,0.592057,0.711531
4,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,3.980689,3.333291,3.988481,3.494274,3.49009,2.370999,2.394222,4.108144,2.68952,...,-0.739483,-0.103737,-1.071226,-0.535766,-1.029838,0.178408,0.361334,-0.453689,0.657613,0.623739
5,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,4.991286,4.222969,4.533385,3.712087,4.184306,3.324107,2.698993,4.135948,2.888685,...,-0.592125,0.2066,-1.019552,-0.606019,-1.187632,0.356565,0.547004,-0.900965,0.880465,0.414951


In [9]:
samples = df_genes.pop("Sample")

In [10]:
df_genes = df_genes.astype(float)

In [11]:
df_genes.iloc[0].dtype

dtype('float64')

# Feature Selection

In [12]:
from bps_numerical.clustering import CorrelationClusterer
from bps_numerical.feature_selection import FirstFeatureSelector

In [13]:
clusterer = CorrelationClusterer(
    list(df_genes.columns),
    cutoff_threshold=0.3,
    debug=False
)

In [14]:
fs = FirstFeatureSelector(clusterer=clusterer)

In [15]:
cols_genes = fs.select_features(df_genes)

2022-07-01 15:08:59.299 | DEBUG    | bps_numerical.clustering:cluster:62 - Computing correlation for (6832, 25000)
2022-07-01 15:09:15.921 | INFO     | bps_numerical.clustering:_cluster:92 - Clustering in progress...
2022-07-01 15:09:35.260 | DEBUG    | bps_numerical.clustering:cluster:86 - Took 19.338802814483643 seconds to form 9268 clusters...


In [16]:
len(cols_genes)

9268

In [17]:
cols_genes[:10]

['ENSMUSG00000099250',
 'ENSMUSG00000064339',
 'ENSMUSG00000064351',
 'ENSMUSG00000029368',
 'ENSMUSG00000098178',
 'ENSMUSG00000037071',
 'ENSMUSG00000106106',
 'ENSMUSG00000105361',
 'ENSMUSG00000020609',
 'ENSMUSG00000032554']

# Data Prep

In [18]:
df_merged = merge_gene_phenotype(
    pd.concat([samples, df_genes[cols_genes]], axis=1),
    CSV_PHENOTYPE,
    "Sample",
)

2022-07-01 15:09:43.985 | INFO     | bps_numerical.preprocessing:merge_gene_phenotype:52 - Merging gene-phenotype dataframes...


In [19]:
df_merged.shape

(6832, 9281)

In [20]:
df_merged.head()

,Sample,age,animalreturn,dataset,condition,duration,gender,libPrep,mission,preservation,...,ENSMUSG00000009585,ENSMUSG00000013921,ENSMUSG00000013611,ENSMUSG00000014554,ENSMUSG00000013523,ENSMUSG00000013418,ENSMUSG00000013415,ENSMUSG00000013353,ENSMUSG00000013158,ENSMUSG00000012819
0,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,10,LAR,GLDS_242,GC,33,male,ribodepleted,RR9,immediate,...,0.582737,-0.595290,-1.150503,0.700040,-1.240282,-0.977355,-1.006752,-1.311184,-0.824456,-1.043170
1,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,16,ISST,GLDS_48,FLT,37,female,polyA,RR1_NASA,carcass,...,0.221626,-0.025088,-1.013243,0.856615,-1.134398,-1.024068,-0.838561,-1.241144,-0.981220,-1.151805
2,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,0.420400,-0.474848,-1.127936,0.684060,-1.114766,-1.004149,-1.074711,-1.138751,-1.053856,-1.094534
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,0.501785,-0.696546,-1.217986,0.700697,-1.137912,-1.112110,-1.107865,-1.070167,-1.217674,-1.029838
4,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,32,LAR,GLDS_245,GC,29,female,ribodepleted,RR6,immediate,...,0.220479,-0.256057,-0.899914,0.755811,-1.138909,-0.896629,-0.961937,-1.059288,-1.152877,-1.187632


In [21]:
cols_phenotype = ["condition"]
cols_to_use = cols_genes + cols_phenotype

In [22]:
df_to_use = df_merged[cols_to_use]

In [23]:
df_to_use[cols_phenotype].value_counts()

condition
FLT          3477
GC           3355
dtype: int64

In [24]:
df_to_use = pd.get_dummies(df_to_use)

In [25]:
df_to_use.shape

(6832, 9270)

In [26]:
df_to_use.head()

,ENSMUSG00000099250,ENSMUSG00000064339,ENSMUSG00000064351,ENSMUSG00000029368,ENSMUSG00000098178,ENSMUSG00000037071,ENSMUSG00000106106,ENSMUSG00000105361,ENSMUSG00000020609,ENSMUSG00000032554,...,ENSMUSG00000013611,ENSMUSG00000014554,ENSMUSG00000013523,ENSMUSG00000013418,ENSMUSG00000013415,ENSMUSG00000013353,ENSMUSG00000013158,ENSMUSG00000012819,condition_FLT,condition_GC
0,4.565105,4.486067,4.137298,4.149449,2.686711,3.704240,1.342527,-0.469961,3.456564,3.473342,...,-1.150503,0.700040,-1.240282,-0.977355,-1.006752,-1.311184,-0.824456,-1.043170,0,1
1,0.563454,3.140386,4.384875,4.180711,2.497359,2.849660,1.280951,-0.976236,2.765367,3.339221,...,-1.013243,0.856615,-1.134398,-1.024068,-0.838561,-1.241144,-0.981220,-1.151805,1,0
2,3.968279,3.490494,3.721062,4.093958,1.943452,3.477409,2.345267,1.763699,3.526118,3.562559,...,-1.127936,0.684060,-1.114766,-1.004149,-1.074711,-1.138751,-1.053856,-1.094534,1,0
3,3.980689,3.333291,3.494274,4.108144,2.104545,3.348338,2.658457,1.892642,3.504901,3.594130,...,-1.217986,0.700697,-1.137912,-1.112110,-1.107865,-1.070167,-1.217674,-1.029838,1,0
4,4.991286,4.222969,3.712087,4.135948,2.535683,3.040980,1.118650,0.347573,3.336408,3.650379,...,-0.899914,0.755811,-1.138909,-0.896629,-0.961937,-1.059288,-1.152877,-1.187632,0,1


In [27]:
labels = list(set(df_to_use.columns) - set(cols_genes) - set(cols_phenotype))
labels

['condition_FLT', 'condition_GC']

# Trainer

In [38]:
from loguru import logger

In [37]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
import xgboost

In [49]:
from typing import List, Type, Dict

In [50]:
import time

In [63]:
class SingePhenotypeTrainer:
    def __init__(
        self, cols_genes: List[str],
        phenotype:str,
        model: Type[BaseEstimator],
        debug: bool = False,
    ) -> None:
        self.cols_genes = list(cols_genes)
        self.phenotype = phenotype
        self.model = model or xgboost.XGBClassifier()
        self.debug = debug
        
    def train(self, data: pd.DataFrame, test_size:float = 0.2):
        target_counts = data[self.phenotype].value_counts()
        data = data[self.cols_genes + [self.phenotype]]
        data = pd.get_dummies(data)
        
        labels = list(set(df_to_use.columns) - set(cols_genes) - set(cols_phenotype))
        
        if self.debug:
            logger.debug(f"Target phenotype stats:: {target_counts}")
            logger.debug(f"n genes = {len(self.cols_genes)} || Labels -> {labels}")
        
        X_train, X_test, Y_train, Y_test = train_test_split(
            data[self.cols_genes], 
            data[labels],
            test_size=test_size,
        )
        tracker = self.fit(X_train, Y_train, X_test, Y_test)
        tracker["labels"] = labels
        return tracker
    
    def fit(self, X_train, Y_train, X_test, Y_test) -> Dict[str, float]:
        start = time.time()
        logger.info(f"Training...")
        
        if self.debug:
            logger.debug(f"X_train={X_train.shape} | Y_train={Y_train.shape} | X_test={X_test.shape} | Y_test={Y_test.shape}")
            logger.debug(f"Modle = {self.model}")
        
        self.model.fit(X_train, Y_train)
        metrics = {
            "train_score": self.model.score(X_train, Y_train),
            "test_score": self.model.score(X_test, Y_test)
        }
        
        end = time.time()
        logger.debug(f"Training took {start-end} seconds.")
        return metrics

In [64]:
model = xgboost.XGBClassifier()
trainer = SingePhenotypeTrainer(
    cols_genes=cols_genes,
    phenotype="condition",
    model = model
)

In [65]:
metrics = trainer.train(df_merged)

2022-07-01 15:32:56.114 | INFO     | __main__:fit:35 - Training...
2022-07-01 15:33:34.102 | DEBUG    | __main__:fit:48 - Training took -37.98729705810547 seconds.


In [66]:
metrics

{'train_score': 1.0,
 'test_score': 1.0,
 'labels': ['condition_FLT', 'condition_GC']}

# Feature Scorer

In [73]:
from abc import ABC, abstractmethod

from typing import List, Tuple, Optional
class FeatureScorer(ABC):
    def __init__(self, model: Type[BaseEstimator]) -> None:
        self.model = model
    
    @abstractmethod
    def get_features(self, *args, **kwargs) -> List[Tuple[str, float]]:
        raise NotImplementedError()

class SinglePhenotypeFeatureScorer(FeatureScorer):
    
    def get_features(self, top_k: int = 100, columns: Optional[List[str]]=None) -> List[Tuple[str, float]]:
        cols = columns or self.model.feature_names_in_
        importances = self.model.feature_importances_
        indices = np.argsort(-importances)[:top_k]
        cols = np.array(cols)[indices].tolist()
        return list(zip(cols, importances[indices].tolist()))

In [74]:
SinglePhenotypeFeatureScorer(trainer.model).get_features(top_k=10)

[('ENSMUSG00000113575', 0.2343384325504303),
 ('ENSMUSG00000056999', 0.11355514824390411),
 ('ENSMUSG00000094320', 0.046360306441783905),
 ('ENSMUSG00000074373', 0.03878649324178696),
 ('ENSMUSG00000073000', 0.030737265944480896),
 ('ENSMUSG00000096688', 0.02965598925948143),
 ('ENSMUSG00000102135', 0.02877301163971424),
 ('ENSMUSG00000090176', 0.02293839305639267),
 ('ENSMUSG00000111528', 0.020082658156752586),
 ('ENSMUSG00000098013', 0.02006441541016102)]